In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
owid_df = pd.read_csv('../data/energy-data/owid-energy-data.csv')

In [3]:
gov_response = pd.read_csv('../data/covid-policy-dataset/data/timeseries_indices/OxCGRT_timeseries_GovernmentResponseIndex_v1.csv')

In [4]:
primary_cons = pd.read_excel('../data/Statistical Review of World Energy Data.xlsx'
                             ,sheet_name='Primary energy cons - EJ'
                             ,skiprows=range(0,2)
                            ,skipfooter=11)

In [5]:
primary_cons.rename(columns={'Exajoules':'Country'}, inplace=True)

In [6]:
gov_response_countries = gov_response[gov_response['Jurisdiction']=='NAT_TOTAL']

In [7]:
print(primary_cons['Country'].to_string())

0                           NaN
1                        Canada
2                        Mexico
3                            US
4           Total North America
5                           NaN
6                     Argentina
7                        Brazil
8                         Chile
9                      Colombia
10                      Ecuador
11                         Peru
12            Trinidad & Tobago
13                    Venezuela
14              Central America
15              Other Caribbean
16          Other South America
17     Total S. & Cent. America
18                          NaN
19                      Austria
20                      Belgium
21                     Bulgaria
22                      Croatia
23                       Cyprus
24               Czech Republic
25                      Denmark
26                      Estonia
27                      Finland
28                       France
29                      Germany
30                       Greece
31      

In [8]:
gov_response_rotate = gov_response_countries[gov_response_countries['CountryName'].isin(owid_df['country'])].drop(columns=['CountryCode'
                                                                                                          ,'RegionCode'
                                                                                                          ,'RegionName'
                                                                                                          ,'CityCode'
                                                                                                          ,'CityName'
                                                                                                          ,'Jurisdiction']).transpose().reset_index()

In [9]:
primary_cons_rotate = primary_cons[primary_cons['Country'].isin(gov_response_countries['CountryName'])][['Country',2019,2020,2021,2022,2023]].reset_index(drop=True).transpose()

In [10]:
primary_cons_rotate.columns = primary_cons_rotate.iloc[0]

In [11]:
primary_cons_rotate = primary_cons_rotate.reset_index().drop(index=0).reset_index(drop=True).rename(columns={'index':'Year'}).apply(pd.to_numeric)

In [12]:
gov_response_rotate.columns = gov_response_rotate.iloc[0]

In [13]:
gov_response_rotate = gov_response_rotate.drop(index=0).rename(columns={'CountryName':'Date'}).reset_index(drop=True)

In [14]:
gov_num = gov_response_rotate.iloc[:,1:73].apply(pd.to_numeric)

In [15]:
gov_date = gov_response_rotate.iloc[:,0].apply(pd.to_datetime)

In [16]:
gov_response_rotate.columns[1:]

Index(['Aruba', 'Afghanistan', 'Angola', 'Albania', 'United Arab Emirates',
       'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Burundi',
       ...
       'United States', 'Uzbekistan', 'Venezuela',
       'United States Virgin Islands', 'Vietnam', 'Vanuatu', 'Yemen',
       'South Africa', 'Zambia', 'Zimbabwe'],
      dtype='object', name=0, length=176)

In [17]:
gov_num.insert(0,'Date',gov_date)

In [18]:
gov_num['Year'] = gov_num['Date'].dt.year

In [19]:
gov_mean = gov_num.groupby('Year').mean().reset_index().drop(columns='Date')

In [20]:
primary_cons_rotate = primary_cons_rotate.reset_index(drop=True)

In [21]:
gov_melt = gov_mean.melt(id_vars=['Year'],value_vars=gov_mean.columns[1:]).rename(columns={0:'Country'})

In [22]:
primary_cons_rotate

Country,Year,Canada,Mexico,Argentina,Brazil,Chile,Colombia,Ecuador,Peru,Venezuela,...,Malaysia,New Zealand,Pakistan,Philippines,Singapore,South Korea,Sri Lanka,Taiwan,Thailand,Vietnam
0,2019,14.644261,8.069090,3.338490,12.718623,1.706323,2.027493,0.760799,1.175707,2.365085,...,4.466813,0.949265,3.537059,2.038037,3.346846,12.465634,0.406927,4.790488,5.327082,4.329488
1,2020,13.758059,7.350903,3.131383,12.219726,1.592920,1.845309,0.658305,1.036308,1.900297,...,4.307145,0.861325,3.517981,1.849511,3.277865,12.049074,0.381602,4.650168,4.952731,4.335817
2,2021,13.921114,7.833220,3.450466,12.854411,1.716685,2.069083,0.765349,1.178669,2.174710,...,4.487938,0.847978,3.901649,1.963837,3.278570,12.623430,0.394695,4.942589,4.992516,4.350794
3,2022,14.299105,8.183102,3.705772,13.435922,1.812619,2.196243,0.801823,1.216668,2.341003,...,4.792605,0.835960,3.595340,2.055580,3.174647,12.752438,0.351276,4.747891,4.996416,4.478643
4,2023,13.949592,8.452957,3.671069,13.873373,1.814643,2.255347,0.843516,1.237765,2.531421,...,4.807117,0.860885,3.372125,2.193169,3.470479,12.434021,0.357512,4.528818,5.006924,4.891435


In [23]:
gov_mean.to_csv('mean_restrictions.csv', index=False)

In [24]:
primary_cons_rotate.to_csv('primary_consumption.csv',index=False)

In [25]:
gov_melt.to_csv('gov_melt.csv',index=False)

In [26]:
gov_melt

,Year,Country,value
0,2020,Aruba,42.321585
1,2021,Aruba,42.214740
2,2022,Aruba,31.981726
3,2023,Aruba,NaN
4,2020,Afghanistan,33.816694
...,...,...,...
283,2023,Haiti,NaN
284,2020,Hungary,44.600109
285,2021,Hungary,53.617068
286,2022,Hungary,28.892521


In [27]:
owid_trim = owid_df[['country','year','coal_cons_per_capita','hydro_energy_per_capita','oil_energy_per_capita','nuclear_energy_per_capita','gas_energy_per_capita']]

In [28]:
owid_trim = owid_trim[(owid_trim['country'].isin(gov_melt['Country']))&(owid_trim['year'].between(2019,2023))].rename(columns={'country':'Country','year':'Year'})

In [29]:
gov_owid_merge = owid_trim.merge(gov_melt,how='outer',on=('Country','Year'))

In [39]:
#gov_owid_merge.to_csv('gov_owid_merge.csv',index=False)

In [35]:
gov_response[gov_response['CountryCode']=='USA']

,CountryCode,CountryName,RegionCode,RegionName,CityCode,CityName,Jurisdiction,01Jan2020,02Jan2020,03Jan2020,...,19Feb2023,20Feb2023,21Feb2023,22Feb2023,23Feb2023,24Feb2023,25Feb2023,26Feb2023,27Feb2023,28Feb2023
334,USA,United States,US_AK,Alaska,NaN,NaN,STATE_TOTAL,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
335,USA,United States,US_AL,Alabama,NaN,NaN,STATE_TOTAL,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
336,USA,United States,US_AR,Arkansas,NaN,NaN,STATE_TOTAL,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337,USA,United States,US_AZ,Arizona,NaN,NaN,STATE_TOTAL,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338,USA,United States,US_CA,California,NaN,NaN,STATE_TOTAL,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339,USA,United States,US_CO,Colorado,NaN,NaN,STATE_TOTAL,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340,USA,United States,US_CT,Connecticut,NaN,NaN,STATE_TOTAL,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
341,USA,United States,US_DC,Washington DC,NaN,NaN,STATE_TOTAL,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
342,USA,United States,US_DE,Delaware,NaN,NaN,STATE_TOTAL,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
343,USA,United States,US_FL,Florida,NaN,NaN,STATE_TOTAL,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
owid_df[owid_df['country']=='United States']

,country,year,iso_code,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
20369,United States,1900,USA,74829905.0,6.139981e+11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20370,United States,1901,USA,76245912.0,6.831060e+11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20371,United States,1902,USA,77698268.0,6.901268e+11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20372,United States,1903,USA,79187838.0,7.237663e+11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20373,United States,1904,USA,80705948.0,7.146218e+11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20488,United States,2019,USA,334319680.0,1.854055e+13,2.780,11.250,1243.930,415.870,172.021,...,2.570,1.071,8.514,59.126,787.816,885.021,295.88,2356.476,7.115,2.964
20489,United States,2020,USA,335942016.0,1.802736e+13,-9.029,-37.549,1126.150,378.321,162.826,...,3.233,1.408,14.214,108.672,896.488,1005.947,337.94,2668.579,8.358,3.641
20490,United States,2021,USA,336997632.0,1.909938e+13,9.917,37.518,1233.954,415.839,160.980,...,3.958,1.674,11.913,103.127,999.615,1122.263,378.20,2966.237,9.105,3.851
20491,United States,2022,USA,338289856.0,1.949317e+13,5.231,21.754,1293.546,437.593,153.271,...,4.784,2.038,14.834,144.094,1143.709,1283.810,434.30,3380.855,10.131,4.315
